In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')



In [2]:
# Tokenize text data using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 100  # Maximum sequence length
X_train_text = tokenizer(train_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
X_test_text = tokenizer(test_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

# Convert labels to numerical format
label_mapping = {'bully-Spam': 0, 'not_bully-Spam': 1, 'bully-Ham': 2, 'not_bully-Ham': 3}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

# Define BERT model
def build_bert_model():
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    bert_model = TFBertModel.from_pretrained("bert-base-uncased")
    bert_outputs = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]

    return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=bert_outputs)

# Build BERT model
bert_model = build_bert_model()

# Combine BERT and Bi-LSTM models
def build_combined_model(bert_model):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    bert_output = bert_model([input_ids, token_type_ids, attention_mask])
    
    # Add Bi-LSTM layer
    bi_lstm_output = Bidirectional(LSTM(128))(bert_output)
    
    output = Dense(4, activation='softmax')(bi_lstm_output)

    return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=output)

# Build combined model
combined_model = build_combined_model(bert_model)

# Compile combined model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed
combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train combined model with a smaller batch size
history = combined_model.fit(
    [X_train_text['input_ids'], X_train_text['token_type_ids'], X_train_text['attention_mask']], 
    y_train, 
    epochs=5, 
    batch_size=16, 
    validation_split=0.1
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/5
79/79 [==============================] - 358s 4s/step - loss: 0.7257 - accuracy: 0.7272 - val_loss: 0.3581 - val_accuracy: 0.9220
Epoch 2/5
79/79 [==============================] - 343s 4s/step - loss: 0.3440 - accuracy: 0.8898 - val_loss: 0.2275 - val_accuracy: 0.9362
Epoch 3/5
79/79 [==============================] - 345s 4s/step - loss: 0.1674 - accuracy: 0.9532 - val_loss: 0.2147 - val_accuracy: 0.9291
Epoch 4/5
79/79 [==============================] - 344s 4s/step - loss: 0.0846 - accuracy: 0.9794 - val_loss: 0.0901 - val_accuracy: 0.9716
Epoch 5/5
79/79 [==============================] - 343s 4s/step - loss: 0.0551 - accuracy: 0.9857 - val_loss: 0.0649 - val_accuracy: 0.9787


In [3]:
# Make predictions on test data
predictions = combined_model.predict([X_test_text['input_ids'], X_test_text['token_type_ids'], X_test_text['attention_mask']])

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the model
from sklearn.metrics import classification_report

# Convert test labels to numpy array
y_test_np = np.array(y_test)

# Print classification report
print(classification_report(y_test_np, predicted_labels))


14/14 [==============================] - 34s 2s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.85      0.92       115
           2       0.83      0.88      0.85       103
           3       0.87      0.91      0.89       198

    accuracy                           0.89       426
   macro avg       0.92      0.91      0.92       426
weighted avg       0.90      0.89      0.89       426

